In [1]:
#r "nuget: Microsoft.ML"
open Microsoft.ML
open System

Installed package Microsoft.ML version *

Create classes to map data to:

In [2]:
[<CLIMutable>]
type SourceData =
    {
        Date: DateTime
    }

[<CLIMutable>]
type MappedData =
    {
        mutable SourceDate: DateTime

        mutable Month: int

        mutable IsWeekend: bool
    }

Instantiate a new MLContext and create some sample data:

In [3]:
let context = new MLContext()

let data =
    [
        { Date = DateTime.Parse("2019-08-20") }
        { Date = DateTime.Parse("2019-07-06") }
        { Date = DateTime.Parse("2019-06-16") }
        { Date = DateTime.Parse("2019-05-14") }
    ]

Create a new DataView from the sample data:

In [4]:
let dataView = context.Data.LoadFromEnumerable<SourceData>(data)

Here is where we define a CustomMapping to map from SourceData to MappedData:

In [5]:
let mapper source mapped =
    mapped.SourceDate <- source.Date
    mapped.Month <- source.Date.Month
    mapped.IsWeekend <- source.Date.DayOfWeek = DayOfWeek.Saturday || source.Date.DayOfWeek = DayOfWeek.Sunday
let mapperAction = new Action<SourceData, MappedData>(mapper)

Fit the encoder to our data:

In [6]:
let encoder = context.Transforms.CustomMapping(mapperAction, "MapDate")

Transform our data using the fitted transformer:

In [7]:
let transformer = encoder.Fit(dataView)

Transform our data using the fitted transformer:

In [8]:
let transformedDataView = transformer.Transform(dataView)

Print our transformed data:

In [9]:
let mappedData = context.Data.CreateEnumerable<MappedData>(transformedDataView, reuseRowObject = true)

mappedData
|> Seq.iter (printfn "%A")

{ SourceDate = 8/20/2019 12:00:00 AM
  Month = 8
  IsWeekend = false }
{ SourceDate = 7/6/2019 12:00:00 AM
  Month = 7
  IsWeekend = true }
{ SourceDate = 6/16/2019 12:00:00 AM
  Month = 6
  IsWeekend = true }
{ SourceDate = 5/14/2019 12:00:00 AM
  Month = 5
  IsWeekend = false }
